In [6]:
import gluonbook as gb
import math
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, rnn
import time
(corpus_indices, char_to_idx, idx_to_char,vocab_size) = gb.load_data_jay_lyrics()

In [7]:
num_hiddens = 256
rnn_layer = rnn.RNN(num_hiddens)
rnn_layer.initialize()

In [8]:
batch_size = 2
state = rnn_layer.begin_state(batch_size=batch_size)
state[0].shape

(1, 2, 256)

In [12]:
num_steps = 35
X = nd.random.uniform(shape=(num_steps, batch_size, vocab_size))
Y, state_new = rnn_layer(X, state)
Y.shape, len(state_new), state_new[0].shape

((35, 2, 256), 1, (1, 2, 256))

In [13]:
class RNNModel(nn.Block):
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.dense = nn.Dense(vocab_size)
    def forward(self, inputs, state):
        # 将输⼊转置成（num_steps，batch_size）后获取one-hot 向量表⽰。
        X = nd.one_hot(inputs.T, self.vocab_size)

        Y, state = self.rnn(X, state)
        # 全连接层会⾸先将Y 的形状变成（num_steps * batch_size，num_hiddens），
        # 它的输出形状为（num_steps * batch_size，vocab_size）。
        output = self.dense(Y.reshape((-1, Y.shape[-1])))
        return output, state
    def begin_state(self, *args, **kwargs):
        return self.rnn.begin_state(*args, **kwargs)

In [14]:
def predict_rnn_gluon(prefix, num_chars, model, vocab_size, ctx, idx_to_char,char_to_idx):
    # 使⽤model 的成员函数来初始化隐藏状态。
    state = model.begin_state(batch_size=1, ctx=ctx)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        X = nd.array([output[-1]], ctx=ctx).reshape((1, 1))
        (Y, state) = model(X, state) # 前向计算不需要传⼊模型参数。
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

In [20]:
ctx = gb.try_gpu()
model = RNNModel(rnn_layer, vocab_size)
model.initialize(force_reinit=True, ctx=ctx)
predict_rnn_gluon('分开', 10, model, vocab_size, ctx, idx_to_char, char_to_idx)

'分开轻婆分毛蟑娇缘几娇功'

In [21]:
def train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,corpus_indices, idx_to_char, char_to_idx,num_epochs, num_steps, lr, clipping_theta,batch_size, pred_period, pred_len, prefixes):
    loss = gloss.SoftmaxCrossEntropyLoss()
    model.initialize(ctx=ctx, force_reinit=True, init=init.Normal(0.01))
    trainer = gluon.Trainer(model.collect_params(), 'sgd',{'learning_rate': lr, 'momentum': 0, 'wd': 0})
    for epoch in range(num_epochs):
        loss_sum, start = 0.0, time.time()
        data_iter = gb.data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx)
        state = model.begin_state(batch_size=batch_size, ctx=ctx)
        for t, (X, Y) in enumerate(data_iter):
            for s in state:
                s.detach()
            with autograd.record():
                (output, state) = model(X, state)
                y = Y.T.reshape((-1,))
                l = loss(output, y).mean()
            l.backward()
            # 梯度裁剪。
            params = [p.data() for p in model.collect_params().values()]
            gb.grad_clipping(params, clipping_theta, ctx)
            trainer.step(1) # 因为已经误差取过均值，梯度不⽤再做平均。
            loss_sum += l.asscalar()
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (epoch + 1, math.exp(loss_sum / (t + 1)), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_gluon(prefix, pred_len, model, vocab_size,ctx, idx_to_char, char_to_idx))

In [22]:
num_epochs, batch_size, lr, clipping_theta = 200, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,corpus_indices, idx_to_char, char_to_idx,num_epochs, num_steps, lr, clipping_theta,batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 79.629996, time 0.49 sec
 - 分开 我不能再爱 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我
 - 不分开 我想你这生写我不能你想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我
epoch 100, perplexity 13.354801, time 0.44 sec
 - 分开 我不儿这想瘦我不你再想躲我的证跳面你走力看着我 别不依旧每日折一枝杨柳望著天 看星星暴风圈的老斑鸠
 - 不分开 我给到这样牵三你的手不放开 爱可不可以笑简的暴平有缓走爱可我 你不着 语沉默么面着我 甩开球我满腔
epoch 150, perplexity 4.075850, time 0.44 sec
 - 分开 我不能这样牵 你 不头我 我拿着 难过 篮的完色主  将能不  我的恨都家 一起看着 走谁的脚 我
 - 不分开 我是 你说 我不 我不 我不 我不  不情我的见你就一场悲剧 我可我这辈子注定一个人演戏 最后再一
epoch 200, perplexity 2.470086, time 0.44 sec
 - 分开 我已儿这样走 静着悄悄默默 藤隔壁不多 几非都没有喝水也的窝 它在灌木丛旁邂逅 一只令它心仪的母斑
 - 不分开 我来到的叹息著西 什么我都做泪到 但壶个人 再来一步热粥 一上几斤池里 快攻抢篮板 除非是乌鸦抢了
